# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    temp1_governor_지역명_숫자.csv → g숫자_지역명

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp1_governor_busan_3.csv", "temp1_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp1_governor_지역명_숫자.csv → g숫자_지역명
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '').replace('temp1_', '')
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub 폴더 자동 CSV 로더 ===\n")

#     print("사용법:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     # 이제 g3_busan, g3_seoul 등을 직접 사용 가능
#     """)

#     print("\n다양한 번호 지원:")
#     print("""
#     # 3회차: temp1_governor_busan_3.csv → g3_busan
#     election_3rd = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/3rd_2002")

#     # 4회차: temp1_governor_seoul_4.csv → g4_seoul
#     election_4th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/4th_2006")

#     # 5회차: temp1_governor_daegu_5.csv → g5_daegu
#     election_5th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/5th_2010")
#     """)


# if __name__ == "__main__":
#     example_usage()

# ### 간단한 사용법

# # 한 줄로 모든 CSV 읽기
# username = "your-github-username"
# repo_name = "korean-elections"
# folder_path = "temp/v1_g/3rd_2002"

# election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# # 사용하기
# busan_df = election_data['g3_busan']
# seoul_df = election_data['g3_seoul']

# # 또는 개별 변수로 생성
# create_individual_variables(election_data, globals())
# # 이제 g3_busan, g3_seoul 직접 사용 가능

In [2]:
import pandas as pd

def show_column_info(df):
    """
    데이터프레임의 열 이름과 인덱스를 확인하는 함수
    """
    print("=== 열 이름과 인덱스 ===")
    for idx, col_name in enumerate(df.columns):
        print(f"{idx}: {col_name}")
    print(f"\n총 열 개수: {len(df.columns)}")
    print(f"총 행 개수: {len(df)}")

def rename_columns_by_index(df, index_or_indices, new_name_or_names):
    """
    인덱스를 통해 열 이름을 변경하는 함수

    Parameters:
    - df: pandas DataFrame
    - index_or_indices: 단일 인덱스(int) 또는 인덱스 리스트([int, int, ...])
    - new_name_or_names: 단일 새 이름(str) 또는 새 이름 리스트([str, str, ...])

    Returns:
    - 열 이름이 변경된 DataFrame
    """
    # 단일 값을 리스트로 변환
    if isinstance(index_or_indices, int):
        indices = [index_or_indices]
    else:
        indices = index_or_indices

    if isinstance(new_name_or_names, str):
        new_names = [new_name_or_names]
    else:
        new_names = new_name_or_names

    # 길이 검증
    if len(indices) != len(new_names):
        raise ValueError(f"인덱스 개수({len(indices)})와 새 이름 개수({len(new_names)})가 일치하지 않습니다.")

    # 인덱스 유효성 검사
    for idx in indices:
        if idx < 0 or idx >= len(df.columns):
            raise ValueError(f"유효하지 않은 인덱스: {idx}. 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 열 이름 변경
    rename_dict = {}
    for idx, new_name in zip(indices, new_names):
        old_name = df.columns[idx]
        rename_dict[old_name] = new_name
        print(f"인덱스 {idx}: '{old_name}' -> '{new_name}'")

    result_df = result_df.rename(columns=rename_dict)

    print(f"\n=== 열 이름 변경 완료 ===")
    print(f"변경된 열 개수: {len(rename_dict)}")

    return result_df

def add_sum_column(df, start_idx, end_idx, new_column_name):
    """
    연속된 인덱스 범위의 열들을 합산하여 새로운 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - start_idx: 시작 인덱스
    - end_idx: 끝 인덱스 (포함)
    - new_column_name: 새로 추가할 열의 이름

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    # 유효성 검사
    if start_idx < 0 or end_idx >= len(df.columns) or start_idx > end_idx:
        raise ValueError(f"유효하지 않은 인덱스 범위: ({start_idx}, {end_idx}). 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 합산할 열 이름들 추출
    columns_to_sum = df.columns[start_idx:end_idx + 1]

    print(f"\n=== 합산할 열들 (인덱스 {start_idx}~{end_idx}) ===")
    for idx, col_name in enumerate(columns_to_sum):
        print(f"{start_idx + idx}: {col_name}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 지정된 열들을 합산하여 새로운 열 추가
    # 숫자가 아닌 값은 0으로 처리
    result_df[new_column_name] = df[columns_to_sum].select_dtypes(include=['number']).sum(axis=1, skipna=True)

    print(f"\n=== 새로운 열 추가 완료 ===")
    print(f"새로운 열 이름: {new_column_name}")
    print(f"총 열 개수: {len(result_df.columns)}")

    return result_df

def add_zero_column(df, column_name):
    """
    DataFrame 마지막에 모든 원소가 0(int)인 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - column_name: 추가할 열의 이름 (str)

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    result_df = df.copy()
    result_df[column_name] = 0
    return result_df

# # 사용 예시
# if __name__ == "__main__":
#     # CSV 파일 읽기
#     df = pd.read_csv('temp1_president_14.csv')

#     # 열 정보 확인
#     show_column_info(df)

#     # 예시 1: 단일 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 1: 단일 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df, 0, '광역시도')

#     # 예시 2: 복수 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 2: 복수 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df_renamed, [1, 2], ['시군구', '유권자수'])

#     # 예시 3: 인덱스 4~7 열들 합산 (득표수 1~4번 후보)
#     print("\n" + "="*50)
#     print("예시 3: 열 합산")
#     print("="*50)
#     result_df = add_sum_column(df_renamed, 4, 7, '상위4후보합계')

#     # 변경된 열 정보 확인
#     print("\n" + "="*50)
#     print("변경된 열 정보")
#     print("="*50)
#     show_column_info(result_df)

#     # 결과 확인 (첫 번째 행)
#     print("\n=== 첫 번째 행 결과 ===")
#     first_row = result_df.iloc[0]
#     print(f"광역시도: {first_row['광역시도']}")
#     print(f"시군구: {first_row['시군구']}")
#     print(f"유권자수: {first_row['유권자수']}")
#     print(f"득표수_1_민주자유당_김영삼: {first_row['득표수_1_민주자유당_김영삼']}")
#     print(f"득표수_2_민주당_김대중: {first_row['득표수_2_민주당_김대중']}")
#     print(f"득표수_3_통일국민당_정주영: {first_row['득표수_3_통일국민당_정주영']}")
#     print(f"득표수_4_새한국당_이종찬: {first_row['득표수_4_새한국당_이종찬']}")
#     print(f"상위4후보합계: {first_row['상위4후보합계']}")

#     # 검증: 수동 계산과 비교
#     manual_sum = (first_row['득표수_1_민주자유당_김영삼'] +
#                   first_row['득표수_2_민주당_김대중'] +
#                   first_row['득표수_3_통일국민당_정주영'] +
#                   first_row['득표수_4_새한국당_이종찬'])

#     print(f"\n수동 계산 결과: {manual_sum}")
#     print(f"함수 계산 결과: {first_row['상위4후보합계']}")
#     print(f"계산 일치: {manual_sum == first_row['상위4후보합계']}")

#     # 결과 데이터프레임 출력 (상위 5행)
#     print("\n=== 결과 데이터프레임 (상위 5행) ===")
#     print(result_df.head())

# Data (v1, 6th_2014)

In [3]:
# 한 줄로 모든 CSV 읽기
username = "sw1kwon"
repo_name = "korean-elections"
folder_path = "temp/v1_g/6th_2014"

governor_6 = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# 개별 변수로 생성
create_individual_variables(governor_6, globals())
# 이제 g6_busan, g6_seoul 직접 사용 가능

✅ temp1_governor_busan_6.csv → g6_busan
✅ temp1_governor_chungbuk_6.csv → g6_chungbuk
✅ temp1_governor_chungnam_6.csv → g6_chungnam
✅ temp1_governor_daegu_6.csv → g6_daegu
✅ temp1_governor_daejeon_6.csv → g6_daejeon
✅ temp1_governor_gangwon_6.csv → g6_gangwon
✅ temp1_governor_gwangju_6.csv → g6_gwangju
✅ temp1_governor_gyeongbuk_6.csv → g6_gyeongbuk
✅ temp1_governor_gyeonggi_6.csv → g6_gyeonggi
✅ temp1_governor_gyeongnam_6.csv → g6_gyeongnam
✅ temp1_governor_incheon_6.csv → g6_incheon
✅ temp1_governor_jeju_6.csv → g6_jeju
✅ temp1_governor_jeonbuk_6.csv → g6_jeonbuk
✅ temp1_governor_jeonnam_6.csv → g6_jeonnam
✅ temp1_governor_sejong_6.csv → g6_sejong
✅ temp1_governor_seoul_6.csv → g6_seoul
✅ temp1_governor_ulsan_6.csv → g6_ulsan
변수 생성: g6_busan
변수 생성: g6_chungbuk
변수 생성: g6_chungnam
변수 생성: g6_daegu
변수 생성: g6_daejeon
변수 생성: g6_gangwon
변수 생성: g6_gwangju
변수 생성: g6_gyeongbuk
변수 생성: g6_gyeonggi
변수 생성: g6_gyeongnam
변수 생성: g6_incheon
변수 생성: g6_jeju
변수 생성: g6_jeonbuk
변수 생성: g6_jeonnam
변수 생성: g6_

## Seoul

In [4]:
g6_seoul.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_정몽준,득표수_2_새정치민주연합_박원순,득표수_3_통합진보당_정태흥,득표수_4_새정치국민의당_홍정식,득표수_계,무효투표수,기권수
0,서울특별시,합계,8441594,4948897,2109869,2752171,23638,17603,4903281,45616,3492697
1,서울특별시,종로구,136925,81113,34599,45020,330,238,80187,926,55812
2,서울특별시,중구,113235,65393,28273,35662,291,262,64488,905,47842
3,서울특별시,용산구,204529,118198,58479,57807,462,365,117113,1085,86331
4,서울특별시,성동구,252308,146873,62376,81645,715,546,145282,1591,105435


In [5]:
show_column_info(g6_seoul)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_정몽준
5: 득표수_2_새정치민주연합_박원순
6: 득표수_3_통합진보당_정태흥
7: 득표수_4_새정치국민의당_홍정식
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 26


### v2.1

In [6]:
g6_seoul_21 = rename_columns_by_index(g6_seoul, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_정몽준' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_박원순' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [7]:
g6_seoul_21 = add_sum_column(g6_seoul_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_정태흥
7: 득표수_4_새정치국민의당_홍정식

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [8]:
g6_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_정태흥,득표수_4_새정치국민의당_홍정식,득표수_계,무효투표수,기권수,그외정당
0,서울특별시,합계,8441594,4948897,2109869,2752171,23638,17603,4903281,45616,3492697,41241
1,서울특별시,종로구,136925,81113,34599,45020,330,238,80187,926,55812,568
2,서울특별시,중구,113235,65393,28273,35662,291,262,64488,905,47842,553


In [9]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_seoul_21 = g6_seoul_21[selected_columns].copy()

g6_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,서울특별시,합계,8441594,4948897,2109869,2752171,41241,4903281,45616,3492697
1,서울특별시,종로구,136925,81113,34599,45020,568,80187,926,55812
2,서울특별시,중구,113235,65393,28273,35662,553,64488,905,47842


In [10]:
g6_seoul_21.to_csv("temp2_1_governor_seoul_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [11]:
g6_seoul_22 = rename_columns_by_index(g6_seoul, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_정몽준' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_박원순' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [12]:
g6_seoul_22 = add_sum_column(g6_seoul_22, 6, 7, '그외정당')
g6_seoul_22 = add_zero_column(g6_seoul_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_정태흥
7: 득표수_4_새정치국민의당_홍정식

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [13]:
g6_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_정태흥,득표수_4_새정치국민의당_홍정식,득표수_계,무효투표수,기권수,그외정당,무소속
0,서울특별시,합계,8441594,4948897,2109869,2752171,23638,17603,4903281,45616,3492697,41241,0
1,서울특별시,종로구,136925,81113,34599,45020,330,238,80187,926,55812,568,0
2,서울특별시,중구,113235,65393,28273,35662,291,262,64488,905,47842,553,0


In [14]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_seoul_22 = g6_seoul_22[selected_columns].copy()

g6_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,서울특별시,합계,8441594,4948897,2109869,2752171,41241,0,4903281,45616,3492697
1,서울특별시,종로구,136925,81113,34599,45020,568,0,80187,926,55812
2,서울특별시,중구,113235,65393,28273,35662,553,0,64488,905,47842


In [15]:
g6_seoul_22.to_csv("temp2_2_governor_seoul_6.csv", index=False, encoding="utf-8-sig")

## Busan

In [16]:
g6_busan.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_서병수,득표수_4_무소속_오거돈,득표수_계,무효투표수,기권수
0,부산광역시,합계,2932179,1629167,797926,777225,1575151,54016,1303012
1,부산광역시,중구,41868,23404,12576,10057,22633,771,18464
2,부산광역시,서구,102694,55778,29893,24088,53981,1797,46916
3,부산광역시,동구,83350,47730,25288,20693,45981,1749,35620
4,부산광역시,영도구,115944,60321,30436,27462,57898,2423,55623


In [17]:
show_column_info(g6_busan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_서병수
5: 득표수_4_무소속_오거돈
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 17


### v2.1 (XXX)

In [18]:
g6_busan_21 = rename_columns_by_index(g6_busan, 4, '보수정당')

인덱스 4: '득표수_1_새누리당_서병수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [19]:
g6_busan_21 = add_sum_column(g6_busan_21, 5, 5, '그외정당')
g6_busan_21 = add_zero_column(g6_busan_21, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_4_무소속_오거돈

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [20]:
g6_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_4_무소속_오거돈,득표수_계,무효투표수,기권수,그외정당,진보정당
0,부산광역시,합계,2932179,1629167,797926,777225,1575151,54016,1303012,777225,0
1,부산광역시,중구,41868,23404,12576,10057,22633,771,18464,10057,0
2,부산광역시,서구,102694,55778,29893,24088,53981,1797,46916,24088,0


In [21]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_busan_21 = g6_busan_21[selected_columns].copy()

g6_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,부산광역시,합계,2932179,1629167,797926,0,777225,1575151,54016,1303012
1,부산광역시,중구,41868,23404,12576,0,10057,22633,771,18464
2,부산광역시,서구,102694,55778,29893,0,24088,53981,1797,46916


In [22]:
g6_busan_21.to_csv("temp2_1_governor_busan_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, XX)

In [23]:
g6_busan_22 = rename_columns_by_index(g6_busan, 4, '보수정당')

인덱스 4: '득표수_1_새누리당_서병수' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [24]:
g6_busan_22 = add_sum_column(g6_busan_22, 5, 5, '무소속')
g6_busan_22 = add_zero_column(g6_busan_22, '진보정당')
g6_busan_22 = add_zero_column(g6_busan_22, '그외정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_4_무소속_오거돈

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 10


In [25]:
g6_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_4_무소속_오거돈,득표수_계,무효투표수,기권수,무소속,진보정당,그외정당
0,부산광역시,합계,2932179,1629167,797926,777225,1575151,54016,1303012,777225,0,0
1,부산광역시,중구,41868,23404,12576,10057,22633,771,18464,10057,0,0
2,부산광역시,서구,102694,55778,29893,24088,53981,1797,46916,24088,0,0


In [26]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_busan_22 = g6_busan_22[selected_columns].copy()

g6_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,부산광역시,합계,2932179,1629167,797926,0,0,777225,1575151,54016,1303012
1,부산광역시,중구,41868,23404,12576,0,0,10057,22633,771,18464
2,부산광역시,서구,102694,55778,29893,0,0,24088,53981,1797,46916


In [27]:
g6_busan_22.to_csv("temp2_2_governor_busan_6.csv", index=False, encoding="utf-8-sig")

## Daegu

In [28]:
g6_daegu.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_권영진,득표수_2_새정치민주연합_김부겸,득표수_3_통합진보당_송영우,득표수_4_정의당_이원준,득표수_5_무소속_이정숙,득표수_계,무효투표수,기권수
0,대구광역시,합계,2012579,1052638,581175,418891,10857,12922,14774,1038619,14019,959941
1,대구광역시,중구,65996,35719,20419,13410,313,464,485,35091,628,30277
2,대구광역시,동구,286747,151471,86217,56920,2065,1724,2214,149140,2331,135276
3,대구광역시,서구,183116,91230,56272,29458,981,1209,1609,89529,1701,91886
4,대구광역시,남구,142345,70076,42488,24071,631,817,1015,69022,1054,72269


In [29]:
show_column_info(g6_daegu)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_권영진
5: 득표수_2_새정치민주연합_김부겸
6: 득표수_3_통합진보당_송영우
7: 득표수_4_정의당_이원준
8: 득표수_5_무소속_이정숙
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 9


### v2.1

In [30]:
g6_daegu_21 = rename_columns_by_index(g6_daegu, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_권영진' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_김부겸' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [31]:
g6_daegu_21 = add_sum_column(g6_daegu_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_통합진보당_송영우
7: 득표수_4_정의당_이원준
8: 득표수_5_무소속_이정숙

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [32]:
g6_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_송영우,득표수_4_정의당_이원준,득표수_5_무소속_이정숙,득표수_계,무효투표수,기권수,그외정당
0,대구광역시,합계,2012579,1052638,581175,418891,10857,12922,14774,1038619,14019,959941,38553
1,대구광역시,중구,65996,35719,20419,13410,313,464,485,35091,628,30277,1262
2,대구광역시,동구,286747,151471,86217,56920,2065,1724,2214,149140,2331,135276,6003


In [33]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_daegu_21 = g6_daegu_21[selected_columns].copy()

g6_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대구광역시,합계,2012579,1052638,581175,418891,38553,1038619,14019,959941
1,대구광역시,중구,65996,35719,20419,13410,1262,35091,628,30277
2,대구광역시,동구,286747,151471,86217,56920,6003,149140,2331,135276


In [34]:
g6_daegu_21.to_csv("temp2_1_governor_daegu_6.csv", index=False, encoding="utf-8-sig")

### v2.2

In [35]:
g6_daegu_22 = rename_columns_by_index(g6_daegu, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_권영진' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_김부겸' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [36]:
g6_daegu_22 = add_sum_column(g6_daegu_22, 6, 7, '그외정당')
g6_daegu_22 = add_sum_column(g6_daegu_22, 8, 8, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_송영우
7: 득표수_4_정의당_이원준

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13

=== 합산할 열들 (인덱스 8~8) ===
8: 득표수_5_무소속_이정숙

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 14


In [37]:
g6_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_송영우,득표수_4_정의당_이원준,득표수_5_무소속_이정숙,득표수_계,무효투표수,기권수,그외정당,무소속
0,대구광역시,합계,2012579,1052638,581175,418891,10857,12922,14774,1038619,14019,959941,23779,14774
1,대구광역시,중구,65996,35719,20419,13410,313,464,485,35091,628,30277,777,485
2,대구광역시,동구,286747,151471,86217,56920,2065,1724,2214,149140,2331,135276,3789,2214


In [38]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_daegu_22 = g6_daegu_22[selected_columns].copy()

g6_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대구광역시,합계,2012579,1052638,581175,418891,23779,14774,1038619,14019,959941
1,대구광역시,중구,65996,35719,20419,13410,777,485,35091,628,30277
2,대구광역시,동구,286747,151471,86217,56920,3789,2214,149140,2331,135276


In [39]:
g6_daegu_22.to_csv("temp2_2_governor_daegu_6.csv", index=False, encoding="utf-8-sig")

## Incheon

In [40]:
g6_incheon.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_유정복,득표수_2_새정치민주연합_송영길,득표수_3_통합진보당_신창현,득표수_계,무효투표수,기권수
0,인천광역시,합계,2319198,1244502,615077,593555,22651,1231283,13219,1074696
1,인천광역시,중구,89767,48565,26324,20788,845,47957,608,41202
2,인천광역시,동구,61693,36534,19573,15605,711,35889,645,25159
3,인천광역시,남구,340427,174258,92490,76984,2854,172328,1930,166169
4,인천광역시,연수구,238453,136706,71942,61389,2154,135485,1221,101747


In [41]:
show_column_info(g6_incheon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_유정복
5: 득표수_2_새정치민주연합_송영길
6: 득표수_3_통합진보당_신창현
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 11


### v2.1

In [42]:
g6_incheon_21 = rename_columns_by_index(g6_incheon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_유정복' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_송영길' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [43]:
g6_incheon_21 = add_sum_column(g6_incheon_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_신창현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [44]:
g6_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_신창현,득표수_계,무효투표수,기권수,그외정당
0,인천광역시,합계,2319198,1244502,615077,593555,22651,1231283,13219,1074696,22651
1,인천광역시,중구,89767,48565,26324,20788,845,47957,608,41202,845
2,인천광역시,동구,61693,36534,19573,15605,711,35889,645,25159,711


In [45]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_incheon_21 = g6_incheon_21[selected_columns].copy()

g6_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,인천광역시,합계,2319198,1244502,615077,593555,22651,1231283,13219,1074696
1,인천광역시,중구,89767,48565,26324,20788,845,47957,608,41202
2,인천광역시,동구,61693,36534,19573,15605,711,35889,645,25159


In [46]:
g6_incheon_21.to_csv("temp2_1_governor_incheon_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [47]:
g6_incheon_22 = rename_columns_by_index(g6_incheon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_유정복' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_송영길' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [48]:
g6_incheon_22 = add_sum_column(g6_incheon_22, 6, 6, '그외정당')
g6_incheon_22 = add_zero_column(g6_incheon_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_신창현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [49]:
g6_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_신창현,득표수_계,무효투표수,기권수,그외정당,무소속
0,인천광역시,합계,2319198,1244502,615077,593555,22651,1231283,13219,1074696,22651,0
1,인천광역시,중구,89767,48565,26324,20788,845,47957,608,41202,845,0
2,인천광역시,동구,61693,36534,19573,15605,711,35889,645,25159,711,0


In [50]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_incheon_22 = g6_incheon_22[selected_columns].copy()

g6_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,인천광역시,합계,2319198,1244502,615077,593555,22651,0,1231283,13219,1074696
1,인천광역시,중구,89767,48565,26324,20788,845,0,47957,608,41202
2,인천광역시,동구,61693,36534,19573,15605,711,0,35889,645,25159


In [51]:
g6_incheon_22.to_csv("temp2_2_governor_incheon_6.csv", index=False, encoding="utf-8-sig")

## Gwangju

In [52]:
g6_gwangju.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이정재,득표수_2_새정치민주연합_윤장현,득표수_3_통합진보당_윤민호,득표수_4_노동당_이병훈,득표수_5_무소속_강운태,득표수_7_무소속_이병완,득표수_계,무효투표수,기권수
0,광주광역시,합계,1138418,650008,21614,367203,21200,6785,201666,16249,634717,15291,488410
1,광주광역시,동구,86465,51253,1889,28605,1238,591,16551,1091,49965,1288,35212
2,광주광역시,서구,242201,142219,4481,82754,3700,1225,43444,3932,139536,2683,99982
3,광주광역시,남구,171997,101811,3160,57010,2357,816,33798,2389,99530,2281,70186
4,광주광역시,북구,351880,198547,6840,113138,6912,2201,59878,4995,193964,4583,153333


In [53]:
show_column_info(g6_gwangju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_이정재
5: 득표수_2_새정치민주연합_윤장현
6: 득표수_3_통합진보당_윤민호
7: 득표수_4_노동당_이병훈
8: 득표수_5_무소속_강운태
9: 득표수_7_무소속_이병완
10: 득표수_계
11: 무효투표수
12: 기권수

총 열 개수: 13
총 행 개수: 6


### v2.1

In [54]:
g6_gwangju_21 = rename_columns_by_index(g6_gwangju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_이정재' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_윤장현' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [55]:
g6_gwangju_21 = add_sum_column(g6_gwangju_21, 6, 9, '그외정당')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_3_통합진보당_윤민호
7: 득표수_4_노동당_이병훈
8: 득표수_5_무소속_강운태
9: 득표수_7_무소속_이병완

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [56]:
g6_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_윤민호,득표수_4_노동당_이병훈,득표수_5_무소속_강운태,득표수_7_무소속_이병완,득표수_계,무효투표수,기권수,그외정당
0,광주광역시,합계,1138418,650008,21614,367203,21200,6785,201666,16249,634717,15291,488410,245900
1,광주광역시,동구,86465,51253,1889,28605,1238,591,16551,1091,49965,1288,35212,19471
2,광주광역시,서구,242201,142219,4481,82754,3700,1225,43444,3932,139536,2683,99982,52301


In [57]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_gwangju_21 = g6_gwangju_21[selected_columns].copy()

g6_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,광주광역시,합계,1138418,650008,21614,367203,245900,634717,15291,488410
1,광주광역시,동구,86465,51253,1889,28605,19471,49965,1288,35212
2,광주광역시,서구,242201,142219,4481,82754,52301,139536,2683,99982


In [58]:
g6_gwangju_21.to_csv("temp2_1_governor_gwangju_6.csv", index=False, encoding="utf-8-sig")

### v2.2

In [59]:
g6_gwangju_22 = rename_columns_by_index(g6_gwangju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_이정재' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_윤장현' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [60]:
g6_gwangju_22 = add_sum_column(g6_gwangju_22, 6, 7, '그외정당')
g6_gwangju_22 = add_sum_column(g6_gwangju_22, 8, 9, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_윤민호
7: 득표수_4_노동당_이병훈

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14

=== 합산할 열들 (인덱스 8~9) ===
8: 득표수_5_무소속_강운태
9: 득표수_7_무소속_이병완

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 15


In [61]:
g6_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_윤민호,득표수_4_노동당_이병훈,득표수_5_무소속_강운태,득표수_7_무소속_이병완,득표수_계,무효투표수,기권수,그외정당,무소속
0,광주광역시,합계,1138418,650008,21614,367203,21200,6785,201666,16249,634717,15291,488410,27985,217915
1,광주광역시,동구,86465,51253,1889,28605,1238,591,16551,1091,49965,1288,35212,1829,17642
2,광주광역시,서구,242201,142219,4481,82754,3700,1225,43444,3932,139536,2683,99982,4925,47376


In [62]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_gwangju_22 = g6_gwangju_22[selected_columns].copy()

g6_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,광주광역시,합계,1138418,650008,21614,367203,27985,217915,634717,15291,488410
1,광주광역시,동구,86465,51253,1889,28605,1829,17642,49965,1288,35212
2,광주광역시,서구,242201,142219,4481,82754,4925,47376,139536,2683,99982


In [63]:
g6_gwangju_22.to_csv("temp2_2_governor_gwangju_6.csv", index=False, encoding="utf-8-sig")

## Daejeon

In [64]:
g6_daejeon.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박성효,득표수_2_새정치민주연합_권선택,득표수_3_통합진보당_김창근,득표수_4_정의당_한창민,득표수_계,무효투표수,기권수
0,대전광역시,합계,1207972,652671,301389,322762,9009,11346,644506,8165,555301
1,대전광역시,동구,202671,104754,51063,48607,1422,1913,103005,1749,97917
2,대전광역시,중구,213239,116220,55956,55596,1209,1754,114515,1705,97019
3,대전광역시,서구,389886,209058,96154,104428,2608,3648,206838,2220,180828
4,대전광역시,유성구,240489,138723,57099,75806,1971,2511,137387,1336,101766


In [65]:
show_column_info(g6_daejeon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_박성효
5: 득표수_2_새정치민주연합_권선택
6: 득표수_3_통합진보당_김창근
7: 득표수_4_정의당_한창민
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1

In [66]:
g6_daejeon_21 = rename_columns_by_index(g6_daejeon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_박성효' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_권선택' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [67]:
g6_daejeon_21 = add_sum_column(g6_daejeon_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_김창근
7: 득표수_4_정의당_한창민

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [68]:
g6_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_김창근,득표수_4_정의당_한창민,득표수_계,무효투표수,기권수,그외정당
0,대전광역시,합계,1207972,652671,301389,322762,9009,11346,644506,8165,555301,20355
1,대전광역시,동구,202671,104754,51063,48607,1422,1913,103005,1749,97917,3335
2,대전광역시,중구,213239,116220,55956,55596,1209,1754,114515,1705,97019,2963


In [69]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_daejeon_21 = g6_daejeon_21[selected_columns].copy()

g6_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대전광역시,합계,1207972,652671,301389,322762,20355,644506,8165,555301
1,대전광역시,동구,202671,104754,51063,48607,3335,103005,1749,97917
2,대전광역시,중구,213239,116220,55956,55596,2963,114515,1705,97019


In [70]:
g6_daejeon_21.to_csv("temp2_1_governor_daejeon_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [71]:
g6_daejeon_22 = rename_columns_by_index(g6_daejeon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_박성효' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_권선택' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [72]:
g6_daejeon_22 = add_sum_column(g6_daejeon_22, 6, 7, '그외정당')
g6_daejeon_22 = add_zero_column(g6_daejeon_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_김창근
7: 득표수_4_정의당_한창민

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [73]:
g6_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_김창근,득표수_4_정의당_한창민,득표수_계,무효투표수,기권수,그외정당,무소속
0,대전광역시,합계,1207972,652671,301389,322762,9009,11346,644506,8165,555301,20355,0
1,대전광역시,동구,202671,104754,51063,48607,1422,1913,103005,1749,97917,3335,0
2,대전광역시,중구,213239,116220,55956,55596,1209,1754,114515,1705,97019,2963,0


In [74]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_daejeon_22 = g6_daejeon_22[selected_columns].copy()

g6_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대전광역시,합계,1207972,652671,301389,322762,20355,0,644506,8165,555301
1,대전광역시,동구,202671,104754,51063,48607,3335,0,103005,1749,97917
2,대전광역시,중구,213239,116220,55956,55596,2963,0,114515,1705,97019


In [75]:
g6_daejeon_22.to_csv("temp2_2_governor_daejeon_6.csv", index=False, encoding="utf-8-sig")

## Ulsan

In [76]:
g6_ulsan.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_김기현,득표수_4_정의당_조승수,득표수_5_노동당_이갑용,득표수_계,무효투표수,기권수
0,울산광역시,합계,912325,511881,306311,123736,38107,468154,43727,400444
1,울산광역시,중구,189218,106699,66129,24913,6912,97954,8745,82519
2,울산광역시,남구,277480,151643,95192,36887,8032,140111,11532,125837
3,울산광역시,동구,141266,81696,44961,17794,10811,73566,8130,59570
4,울산광역시,북구,139347,79777,42353,24380,6698,73431,6346,59570


In [77]:
show_column_info(g6_ulsan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_김기현
5: 득표수_4_정의당_조승수
6: 득표수_5_노동당_이갑용
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 6


### v2.1 (XXX)

In [78]:
g6_ulsan_21 = rename_columns_by_index(g6_ulsan, 4, '보수정당')

인덱스 4: '득표수_1_새누리당_김기현' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [79]:
g6_ulsan_21 = add_sum_column(g6_ulsan_21, 5, 6, '그외정당')
g6_ulsan_21 = add_zero_column(g6_ulsan_21, '진보정당')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_4_정의당_조승수
6: 득표수_5_노동당_이갑용

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [80]:
g6_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_4_정의당_조승수,득표수_5_노동당_이갑용,득표수_계,무효투표수,기권수,그외정당,진보정당
0,울산광역시,합계,912325,511881,306311,123736,38107,468154,43727,400444,161843,0
1,울산광역시,중구,189218,106699,66129,24913,6912,97954,8745,82519,31825,0
2,울산광역시,남구,277480,151643,95192,36887,8032,140111,11532,125837,44919,0


In [81]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_ulsan_21 = g6_ulsan_21[selected_columns].copy()

g6_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,울산광역시,합계,912325,511881,306311,0,161843,468154,43727,400444
1,울산광역시,중구,189218,106699,66129,0,31825,97954,8745,82519
2,울산광역시,남구,277480,151643,95192,0,44919,140111,11532,125837


In [82]:
g6_ulsan_21.to_csv("temp2_1_governor_ulsan_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, X)

In [83]:
g6_ulsan_22 = rename_columns_by_index(g6_ulsan, 4, '보수정당')

인덱스 4: '득표수_1_새누리당_김기현' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [84]:
g6_ulsan_22 = add_sum_column(g6_ulsan_22, 5, 6, '그외정당')
g6_ulsan_22 = add_zero_column(g6_ulsan_22, '진보정당')
g6_ulsan_22 = add_zero_column(g6_ulsan_22, '무소속')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_4_정의당_조승수
6: 득표수_5_노동당_이갑용

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [85]:
g6_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_4_정의당_조승수,득표수_5_노동당_이갑용,득표수_계,무효투표수,기권수,그외정당,진보정당,무소속
0,울산광역시,합계,912325,511881,306311,123736,38107,468154,43727,400444,161843,0,0
1,울산광역시,중구,189218,106699,66129,24913,6912,97954,8745,82519,31825,0,0
2,울산광역시,남구,277480,151643,95192,36887,8032,140111,11532,125837,44919,0,0


In [86]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_ulsan_22 = g6_ulsan_22[selected_columns].copy()

g6_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,울산광역시,합계,912325,511881,306311,0,161843,0,468154,43727,400444
1,울산광역시,중구,189218,106699,66129,0,31825,0,97954,8745,82519
2,울산광역시,남구,277480,151643,95192,0,44919,0,140111,11532,125837


In [87]:
g6_ulsan_22.to_csv("temp2_2_governor_ulsan_6.csv", index=False, encoding="utf-8-sig")

## Sejong

In [88]:
g6_sejong.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_유한식,득표수_2_새정치민주연합_이춘희,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,101559,63629,26451,36203,62654,975,37930


In [89]:
show_column_info(g6_sejong)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_유한식
5: 득표수_2_새정치민주연합_이춘희
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 1


### v2.1 (XX)

In [90]:
g6_sejong_21 = rename_columns_by_index(g6_sejong, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_유한식' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_이춘희' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [91]:
g6_sejong_21 = add_zero_column(g6_sejong_21, '그외정당')

In [92]:
g6_sejong_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당
0,세종특별자치시,합계,101559,63629,26451,36203,62654,975,37930,0


In [93]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_sejong_21 = g6_sejong_21[selected_columns].copy()

g6_sejong_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,101559,63629,26451,36203,0,62654,975,37930


In [94]:
g6_sejong_21.to_csv("temp2_1_governor_sejong_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [95]:
g6_sejong_22 = rename_columns_by_index(g6_sejong, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_유한식' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_이춘희' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [96]:
g6_sejong_22 = add_zero_column(g6_sejong_22, '그외정당')
g6_sejong_22 = add_zero_column(g6_sejong_22, '무소속')

In [97]:
g6_sejong_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,세종특별자치시,합계,101559,63629,26451,36203,62654,975,37930,0,0


In [98]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_sejong_22 = g6_sejong_22[selected_columns].copy()

g6_sejong_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,101559,63629,26451,36203,0,0,62654,975,37930


In [99]:
g6_sejong_22.to_csv("temp2_2_governor_sejong_6.csv", index=False, encoding="utf-8-sig")

## Gyeonggi

In [100]:
g6_gyeonggi.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_남경필,득표수_2_새정치민주연합_김진표,득표수_계,무효투표수,기권수
0,경기도,합계,9679317,5156691,2524981,2481824,5006805,149886,4522626
1,경기도,수원시장안구,236664,135841,64235,68152,132387,3454,100823
2,경기도,수원시권선구,230904,121183,58441,59901,118342,2841,109721
3,경기도,수원시팔달구,203541,105744,54133,48962,103095,2649,97797
4,경기도,수원시영통구,240776,142435,58276,81424,139700,2735,98341


In [101]:
show_column_info(g6_gyeonggi)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_남경필
5: 득표수_2_새정치민주연합_김진표
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 45


### v2.1 (XX)

In [102]:
g6_gyeonggi_21 = rename_columns_by_index(g6_gyeonggi, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_남경필' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_김진표' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [103]:
g6_gyeonggi_21 = add_zero_column(g6_gyeonggi_21, '그외정당')

In [104]:
g6_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당
0,경기도,합계,9679317,5156691,2524981,2481824,5006805,149886,4522626,0
1,경기도,수원시장안구,236664,135841,64235,68152,132387,3454,100823,0
2,경기도,수원시권선구,230904,121183,58441,59901,118342,2841,109721,0


In [105]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_gyeonggi_21 = g6_gyeonggi_21[selected_columns].copy()

g6_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경기도,합계,9679317,5156691,2524981,2481824,0,5006805,149886,4522626
1,경기도,수원시장안구,236664,135841,64235,68152,0,132387,3454,100823
2,경기도,수원시권선구,230904,121183,58441,59901,0,118342,2841,109721


In [106]:
g6_gyeonggi_21.to_csv("temp2_1_governor_gyeonggi_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [107]:
g6_gyeonggi_22 = rename_columns_by_index(g6_gyeonggi, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_남경필' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_김진표' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [108]:
g6_gyeonggi_22 = add_zero_column(g6_gyeonggi_22, '그외정당')
g6_gyeonggi_22 = add_zero_column(g6_gyeonggi_22, '무소속')

In [109]:
g6_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,경기도,합계,9679317,5156691,2524981,2481824,5006805,149886,4522626,0,0
1,경기도,수원시장안구,236664,135841,64235,68152,132387,3454,100823,0,0
2,경기도,수원시권선구,230904,121183,58441,59901,118342,2841,109721,0,0


In [110]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_gyeonggi_22 = g6_gyeonggi_22[selected_columns].copy()

g6_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경기도,합계,9679317,5156691,2524981,2481824,0,0,5006805,149886,4522626
1,경기도,수원시장안구,236664,135841,64235,68152,0,0,132387,3454,100823
2,경기도,수원시권선구,230904,121183,58441,59901,0,0,118342,2841,109721


In [111]:
g6_gyeonggi_22.to_csv("temp2_2_governor_gyeonggi_6.csv", index=False, encoding="utf-8-sig")

## Gangwon

In [112]:
g6_gangwon.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_최흥집,득표수_2_새정치민주연합_최문순,득표수_3_통합진보당_이승재,득표수_계,무효투표수,기권수
0,강원도,합계,1255469,781359,369201,381338,15774,766313,15046,474110
1,강원도,춘천시,220197,131007,50016,77737,1857,129610,1397,89190
2,강원도,원주시,257088,142997,64877,73995,2442,141314,1683,114091
3,강원도,강릉시,176430,104326,60251,40526,1736,102513,1813,72104
4,강원도,동해시,75796,45629,23041,20786,942,44769,860,30167


In [113]:
show_column_info(g6_gangwon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_최흥집
5: 득표수_2_새정치민주연합_최문순
6: 득표수_3_통합진보당_이승재
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 18


### v2.1

In [114]:
g6_gangwon_21 = rename_columns_by_index(g6_gangwon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_최흥집' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_최문순' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [115]:
g6_gangwon_21 = add_sum_column(g6_gangwon_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_이승재

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [116]:
g6_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_이승재,득표수_계,무효투표수,기권수,그외정당
0,강원도,합계,1255469,781359,369201,381338,15774,766313,15046,474110,15774
1,강원도,춘천시,220197,131007,50016,77737,1857,129610,1397,89190,1857
2,강원도,원주시,257088,142997,64877,73995,2442,141314,1683,114091,2442


In [117]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_gangwon_21 = g6_gangwon_21[selected_columns].copy()

g6_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,강원도,합계,1255469,781359,369201,381338,15774,766313,15046,474110
1,강원도,춘천시,220197,131007,50016,77737,1857,129610,1397,89190
2,강원도,원주시,257088,142997,64877,73995,2442,141314,1683,114091


In [118]:
g6_gangwon_21.to_csv("temp2_1_governor_gangwon_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [119]:
g6_gangwon_22 = rename_columns_by_index(g6_gangwon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_최흥집' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_최문순' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [120]:
g6_gangwon_22 = add_sum_column(g6_gangwon_22, 6, 6, '그외정당')
g6_gangwon_22 = add_zero_column(g6_gangwon_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_이승재

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [121]:
g6_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_이승재,득표수_계,무효투표수,기권수,그외정당,무소속
0,강원도,합계,1255469,781359,369201,381338,15774,766313,15046,474110,15774,0
1,강원도,춘천시,220197,131007,50016,77737,1857,129610,1397,89190,1857,0
2,강원도,원주시,257088,142997,64877,73995,2442,141314,1683,114091,2442,0


In [122]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_gangwon_22 = g6_gangwon_22[selected_columns].copy()

g6_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,강원도,합계,1255469,781359,369201,381338,15774,0,766313,15046,474110
1,강원도,춘천시,220197,131007,50016,77737,1857,0,129610,1397,89190
2,강원도,원주시,257088,142997,64877,73995,2442,0,141314,1683,114091


In [123]:
g6_gangwon_22.to_csv("temp2_2_governor_gangwon_6.csv", index=False, encoding="utf-8-sig")

## Chungbuk

In [124]:
g6_chungbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_윤진식,득표수_2_새정치민주연합_이시종,득표수_3_통합진보당_신장호,득표수_계,무효투표수,기권수
0,충청북도,합계,1261119,741049,346152,361115,18590,725857,15192,520070
1,충청북도,청주시상당구,194201,107458,51082,52717,2382,106181,1277,86743
2,충청북도,청주시흥덕구,328448,178070,79126,92795,4259,176180,1890,150378
3,충청북도,충주시,169145,97900,49606,45133,1571,96310,1590,71245
4,충청북도,제천시,112013,67732,31934,32605,1619,66158,1574,44281


In [125]:
show_column_info(g6_chungbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_윤진식
5: 득표수_2_새정치민주연합_이시종
6: 득표수_3_통합진보당_신장호
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 14


### v2.1

In [126]:
g6_chungbuk_21 = rename_columns_by_index(g6_chungbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_윤진식' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_이시종' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [127]:
g6_chungbuk_21 = add_sum_column(g6_chungbuk_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_신장호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [128]:
g6_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_신장호,득표수_계,무효투표수,기권수,그외정당
0,충청북도,합계,1261119,741049,346152,361115,18590,725857,15192,520070,18590
1,충청북도,청주시상당구,194201,107458,51082,52717,2382,106181,1277,86743,2382
2,충청북도,청주시흥덕구,328448,178070,79126,92795,4259,176180,1890,150378,4259


In [129]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_chungbuk_21 = g6_chungbuk_21[selected_columns].copy()

g6_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청북도,합계,1261119,741049,346152,361115,18590,725857,15192,520070
1,충청북도,청주시상당구,194201,107458,51082,52717,2382,106181,1277,86743
2,충청북도,청주시흥덕구,328448,178070,79126,92795,4259,176180,1890,150378


In [130]:
g6_chungbuk_21.to_csv("temp2_1_governor_chungbuk_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [131]:
g6_chungbuk_22 = rename_columns_by_index(g6_chungbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_윤진식' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_이시종' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [132]:
g6_chungbuk_22 = add_sum_column(g6_chungbuk_22, 6, 6, '그외정당')
g6_chungbuk_22 = add_zero_column(g6_chungbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_신장호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [133]:
g6_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_신장호,득표수_계,무효투표수,기권수,그외정당,무소속
0,충청북도,합계,1261119,741049,346152,361115,18590,725857,15192,520070,18590,0
1,충청북도,청주시상당구,194201,107458,51082,52717,2382,106181,1277,86743,2382,0
2,충청북도,청주시흥덕구,328448,178070,79126,92795,4259,176180,1890,150378,4259,0


In [134]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_chungbuk_22 = g6_chungbuk_22[selected_columns].copy()

g6_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청북도,합계,1261119,741049,346152,361115,18590,0,725857,15192,520070
1,충청북도,청주시상당구,194201,107458,51082,52717,2382,0,106181,1277,86743
2,충청북도,청주시흥덕구,328448,178070,79126,92795,4259,0,176180,1890,150378


In [135]:
g6_chungbuk_22.to_csv("temp2_2_governor_chungbuk_6.csv", index=False, encoding="utf-8-sig")

## Chungnam

In [136]:
g6_chungnam.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_정진석,득표수_2_새정치민주연합_안희정,득표수_4_무소속_김기문,득표수_계,무효투표수,기권수
0,충청남도,합계,1644554,916206,392315,465994,34204,892513,23693,728348
1,충청남도,천안시서북구,223113,103339,40623,58379,3198,102200,1139,119774
2,충청남도,천안시동남구,236496,117049,49771,62796,2987,115554,1495,119447
3,충청남도,공주시,96439,57596,26855,27730,1519,56104,1492,38843
4,충청남도,보령시,86603,55365,25254,25451,2550,53255,2110,31238


In [137]:
show_column_info(g6_chungnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_정진석
5: 득표수_2_새정치민주연합_안희정
6: 득표수_4_무소속_김기문
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 17


### v2.1

In [138]:
g6_chungnam_21 = rename_columns_by_index(g6_chungnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_정진석' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_안희정' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [139]:
g6_chungnam_21 = add_sum_column(g6_chungnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_무소속_김기문

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [140]:
g6_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_4_무소속_김기문,득표수_계,무효투표수,기권수,그외정당
0,충청남도,합계,1644554,916206,392315,465994,34204,892513,23693,728348,34204
1,충청남도,천안시서북구,223113,103339,40623,58379,3198,102200,1139,119774,3198
2,충청남도,천안시동남구,236496,117049,49771,62796,2987,115554,1495,119447,2987


In [141]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_chungnam_21 = g6_chungnam_21[selected_columns].copy()

g6_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청남도,합계,1644554,916206,392315,465994,34204,892513,23693,728348
1,충청남도,천안시서북구,223113,103339,40623,58379,3198,102200,1139,119774
2,충청남도,천안시동남구,236496,117049,49771,62796,2987,115554,1495,119447


In [142]:
g6_chungnam_21.to_csv("temp2_1_governor_chungnam_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX)

In [143]:
g6_chungnam_22 = rename_columns_by_index(g6_chungnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_정진석' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_안희정' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [144]:
g6_chungnam_22 = add_sum_column(g6_chungnam_22, 6, 6, '무소속')
g6_chungnam_22 = add_zero_column(g6_chungnam_22, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_무소속_김기문

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 11


In [145]:
g6_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_4_무소속_김기문,득표수_계,무효투표수,기권수,무소속,그외정당
0,충청남도,합계,1644554,916206,392315,465994,34204,892513,23693,728348,34204,0
1,충청남도,천안시서북구,223113,103339,40623,58379,3198,102200,1139,119774,3198,0
2,충청남도,천안시동남구,236496,117049,49771,62796,2987,115554,1495,119447,2987,0


In [146]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_chungnam_22 = g6_chungnam_22[selected_columns].copy()

g6_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청남도,합계,1644554,916206,392315,465994,0,34204,892513,23693,728348
1,충청남도,천안시서북구,223113,103339,40623,58379,0,3198,102200,1139,119774
2,충청남도,천안시동남구,236496,117049,49771,62796,0,2987,115554,1495,119447


In [147]:
g6_chungnam_22.to_csv("temp2_2_governor_chungnam_6.csv", index=False, encoding="utf-8-sig")

## Jeonbuk

In [148]:
g6_jeonbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박철곤,득표수_2_새정치민주연합_송하진,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수
0,전라북도,합계,1503242,900029,177172,599654,89337,866163,33866,603213
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,156692,3149,124031
2,전라북도,전주시덕진구,221516,123206,29005,77702,14026,120733,2473,98310
3,전라북도,군산시,220810,117779,19923,83123,10831,113877,3902,103031
4,전라북도,익산시,244122,129480,22697,91000,11226,124923,4557,114642


In [149]:
show_column_info(g6_jeonbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_박철곤
5: 득표수_2_새정치민주연합_송하진
6: 득표수_3_통합진보당_이광석
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 16


### v2.1

In [150]:
g6_jeonbuk_21 = rename_columns_by_index(g6_jeonbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_박철곤' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_송하진' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [151]:
g6_jeonbuk_21 = add_sum_column(g6_jeonbuk_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_이광석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [152]:
g6_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수,그외정당
0,전라북도,합계,1503242,900029,177172,599654,89337,866163,33866,603213,89337
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,156692,3149,124031,17298
2,전라북도,전주시덕진구,221516,123206,29005,77702,14026,120733,2473,98310,14026


In [153]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_jeonbuk_21 = g6_jeonbuk_21[selected_columns].copy()

g6_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라북도,합계,1503242,900029,177172,599654,89337,866163,33866,603213
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,156692,3149,124031
2,전라북도,전주시덕진구,221516,123206,29005,77702,14026,120733,2473,98310


In [154]:
g6_jeonbuk_21.to_csv("temp2_1_governor_jeonbuk_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [155]:
g6_jeonbuk_22 = rename_columns_by_index(g6_jeonbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_박철곤' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_송하진' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [156]:
g6_jeonbuk_22 = add_sum_column(g6_jeonbuk_22, 6, 6, '그외정당')
g6_jeonbuk_22 = add_zero_column(g6_jeonbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_이광석

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [157]:
g6_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라북도,합계,1503242,900029,177172,599654,89337,866163,33866,603213,89337,0
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,156692,3149,124031,17298,0
2,전라북도,전주시덕진구,221516,123206,29005,77702,14026,120733,2473,98310,14026,0


In [158]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_jeonbuk_22 = g6_jeonbuk_22[selected_columns].copy()

g6_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라북도,합계,1503242,900029,177172,599654,89337,0,866163,33866,603213
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,0,156692,3149,124031
2,전라북도,전주시덕진구,221516,123206,29005,77702,14026,0,120733,2473,98310


In [159]:
g6_jeonbuk_22.to_csv("temp2_2_governor_jeonbuk_6.csv", index=False, encoding="utf-8-sig")

## Jeonnam

In [160]:
g6_jeonnam.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이중효,득표수_2_새정치민주연합_이낙연,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수
0,전라남도,합계,1549440,1015688,92549,755233,120868,968650,47038,533752
1,전라남도,목포시,185570,103181,8017,80860,10840,99717,3464,82389
2,전라남도,여수시,232635,135390,11394,104927,15608,131929,3461,97245
3,전라남도,순천시,214889,126320,10695,95005,16777,122477,3843,88569
4,전라남도,나주시,76122,51515,4157,38556,6067,48780,2735,24607


In [161]:
show_column_info(g6_jeonnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_이중효
5: 득표수_2_새정치민주연합_이낙연
6: 득표수_3_통합진보당_이성수
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 23


### v2.1

In [162]:
g6_jeonnam_21 = rename_columns_by_index(g6_jeonnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_이중효' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_이낙연' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [163]:
g6_jeonnam_21 = add_sum_column(g6_jeonnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_이성수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [164]:
g6_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수,그외정당
0,전라남도,합계,1549440,1015688,92549,755233,120868,968650,47038,533752,120868
1,전라남도,목포시,185570,103181,8017,80860,10840,99717,3464,82389,10840
2,전라남도,여수시,232635,135390,11394,104927,15608,131929,3461,97245,15608


In [165]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_jeonnam_21 = g6_jeonnam_21[selected_columns].copy()

g6_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라남도,합계,1549440,1015688,92549,755233,120868,968650,47038,533752
1,전라남도,목포시,185570,103181,8017,80860,10840,99717,3464,82389
2,전라남도,여수시,232635,135390,11394,104927,15608,131929,3461,97245


In [166]:
g6_jeonnam_21.to_csv("temp2_1_governor_jeonnam_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [167]:
g6_jeonnam_22 = rename_columns_by_index(g6_jeonnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_이중효' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_이낙연' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [168]:
g6_jeonnam_22 = add_sum_column(g6_jeonnam_22, 6, 6, '그외정당')
g6_jeonnam_22 = add_zero_column(g6_jeonnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_이성수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [169]:
g6_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수,그외정당,무소속
0,전라남도,합계,1549440,1015688,92549,755233,120868,968650,47038,533752,120868,0
1,전라남도,목포시,185570,103181,8017,80860,10840,99717,3464,82389,10840,0
2,전라남도,여수시,232635,135390,11394,104927,15608,131929,3461,97245,15608,0


In [170]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_jeonnam_22 = g6_jeonnam_22[selected_columns].copy()

g6_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라남도,합계,1549440,1015688,92549,755233,120868,0,968650,47038,533752
1,전라남도,목포시,185570,103181,8017,80860,10840,0,99717,3464,82389
2,전라남도,여수시,232635,135390,11394,104927,15608,0,131929,3461,97245


In [171]:
g6_jeonnam_22.to_csv("temp2_2_governor_jeonnam_6.csv", index=False, encoding="utf-8-sig")

## Gyeongbuk

In [172]:
g6_gyeongbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_김관용,득표수_2_새정치민주연합_오중기,득표수_3_통합진보당_윤병태,득표수_4_정의당_박창호,득표수_계,무효투표수,기권수
0,경상북도,합계,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809
1,경상북도,포항시북구,216200,112449,82655,20064,2017,5017,109753,2696,103751
2,경상북도,포항시남구,201120,103003,76273,18395,2038,3840,100546,2457,98117
3,경상북도,울릉군,9344,7490,5925,847,158,242,7172,318,1854
4,경상북도,경주시,216922,126290,94824,16705,4915,5391,121835,4455,90632


In [173]:
show_column_info(g6_gyeongbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_김관용
5: 득표수_2_새정치민주연합_오중기
6: 득표수_3_통합진보당_윤병태
7: 득표수_4_정의당_박창호
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 25


### v2.1

In [174]:
g6_gyeongbuk_21 = rename_columns_by_index(g6_gyeongbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_김관용' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_오중기' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [175]:
g6_gyeongbuk_21 = add_sum_column(g6_gyeongbuk_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_윤병태
7: 득표수_4_정의당_박창호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [176]:
g6_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_윤병태,득표수_4_정의당_박창호,득표수_계,무효투표수,기권수,그외정당
0,경상북도,합계,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809,93067
1,경상북도,포항시북구,216200,112449,82655,20064,2017,5017,109753,2696,103751,7034
2,경상북도,포항시남구,201120,103003,76273,18395,2038,3840,100546,2457,98117,5878


In [177]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_gyeongbuk_21 = g6_gyeongbuk_21[selected_columns].copy()

g6_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상북도,합계,2211734,1314925,986989,189603,93067,1269659,45266,896809
1,경상북도,포항시북구,216200,112449,82655,20064,7034,109753,2696,103751
2,경상북도,포항시남구,201120,103003,76273,18395,5878,100546,2457,98117


In [178]:
g6_gyeongbuk_21.to_csv("temp2_1_governor_gyeongbuk_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [179]:
g6_gyeongbuk_22 = rename_columns_by_index(g6_gyeongbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_김관용' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_오중기' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [180]:
g6_gyeongbuk_22 = add_sum_column(g6_gyeongbuk_22, 6, 7, '그외정당')
g6_gyeongbuk_22 = add_zero_column(g6_gyeongbuk_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_윤병태
7: 득표수_4_정의당_박창호

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [181]:
g6_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_윤병태,득표수_4_정의당_박창호,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상북도,합계,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809,93067,0
1,경상북도,포항시북구,216200,112449,82655,20064,2017,5017,109753,2696,103751,7034,0
2,경상북도,포항시남구,201120,103003,76273,18395,2038,3840,100546,2457,98117,5878,0


In [182]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_gyeongbuk_22 = g6_gyeongbuk_22[selected_columns].copy()

g6_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상북도,합계,2211734,1314925,986989,189603,93067,0,1269659,45266,896809
1,경상북도,포항시북구,216200,112449,82655,20064,7034,0,109753,2696,103751
2,경상북도,포항시남구,201120,103003,76273,18395,5878,0,100546,2457,98117


In [183]:
g6_gyeongbuk_22.to_csv("temp2_2_governor_gyeongbuk_6.csv", index=False, encoding="utf-8-sig")

## Gyeongnam

In [184]:
g6_gyeongnam.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_홍준표,득표수_2_새정치민주연합_김경수,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수
0,경상남도,합계,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694
2,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893
3,경상남도,창원시마산합포구,151800,87780,58522,24933,2756,86211,1569,64020
4,경상남도,창원시마산회원구,175918,102205,63561,33392,3673,100626,1579,73713


In [185]:
show_column_info(g6_gyeongnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_홍준표
5: 득표수_2_새정치민주연합_김경수
6: 득표수_3_통합진보당_강병기
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 23


### v2.1

In [186]:
g6_gyeongnam_21 = rename_columns_by_index(g6_gyeongnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_홍준표' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_김경수' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [187]:
g6_gyeongnam_21 = add_sum_column(g6_gyeongnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_강병기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [188]:
g6_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수,그외정당
0,경상남도,합계,2658347,1589673,913162,559367,79015,1551544,38129,1068674,79015
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694,6020
2,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893,7033


In [189]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_gyeongnam_21 = g6_gyeongnam_21[selected_columns].copy()

g6_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상남도,합계,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694
2,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893


In [190]:
g6_gyeongnam_21.to_csv("temp2_1_governor_gyeongnam_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [191]:
g6_gyeongnam_22 = rename_columns_by_index(g6_gyeongnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_홍준표' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_김경수' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [192]:
g6_gyeongnam_22 = add_sum_column(g6_gyeongnam_22, 6, 6, '그외정당')
g6_gyeongnam_22 = add_zero_column(g6_gyeongnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_통합진보당_강병기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [193]:
g6_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상남도,합계,2658347,1589673,913162,559367,79015,1551544,38129,1068674,79015,0
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694,6020,0
2,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893,7033,0


In [194]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_gyeongnam_22 = g6_gyeongnam_22[selected_columns].copy()

g6_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상남도,합계,2658347,1589673,913162,559367,79015,0,1551544,38129,1068674
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,0,112716,2113,89694
2,경상남도,창원시성산구,185723,111830,50723,52430,7033,0,110186,1644,73893


In [195]:
g6_gyeongnam_22.to_csv("temp2_2_governor_gyeongnam_6.csv", index=False, encoding="utf-8-sig")

## Jeju

In [196]:
g6_jeju.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_원희룡,득표수_2_새정치민주연합_신구범,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,467182,293323,172793,99493,12209,3637,288132,5191,173859
1,제주특별자치도,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380
2,제주특별자치도,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479


In [197]:
show_column_info(g6_jeju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_새누리당_원희룡
5: 득표수_2_새정치민주연합_신구범
6: 득표수_3_통합진보당_고승완
7: 득표수_4_새정치국민의당_주종근
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 3


### v2.1

In [198]:
g6_jeju_21 = rename_columns_by_index(g6_jeju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_원희룡' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_신구범' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [199]:
g6_jeju_21 = add_sum_column(g6_jeju_21, 6, 7, '그외정당')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_고승완
7: 득표수_4_새정치국민의당_주종근

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [200]:
g6_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수,그외정당
0,제주특별자치도,합계,467182,293323,172793,99493,12209,3637,288132,5191,173859,15846
1,제주특별자치도,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380,11314
2,제주특별자치도,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479,4532


In [201]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g6_jeju_21 = g6_jeju_21[selected_columns].copy()

g6_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,467182,293323,172793,99493,15846,288132,5191,173859
1,제주특별자치도,제주시,340604,208224,119758,73632,11314,204704,3520,132380
2,제주특별자치도,서귀포시,126578,85099,53035,25861,4532,83428,1671,41479


In [202]:
g6_jeju_21.to_csv("temp2_1_governor_jeju_6.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [203]:
g6_jeju_22 = rename_columns_by_index(g6_jeju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_새누리당_원희룡' -> '보수정당'
인덱스 5: '득표수_2_새정치민주연합_신구범' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [204]:
g6_jeju_22 = add_sum_column(g6_jeju_22, 6, 7, '그외정당')
g6_jeju_22 = add_zero_column(g6_jeju_22, '무소속')


=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_3_통합진보당_고승완
7: 득표수_4_새정치국민의당_주종근

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [205]:
g6_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수,그외정당,무소속
0,제주특별자치도,합계,467182,293323,172793,99493,12209,3637,288132,5191,173859,15846,0
1,제주특별자치도,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380,11314,0
2,제주특별자치도,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479,4532,0


In [206]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g6_jeju_22 = g6_jeju_22[selected_columns].copy()

g6_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,467182,293323,172793,99493,15846,0,288132,5191,173859
1,제주특별자치도,제주시,340604,208224,119758,73632,11314,0,204704,3520,132380
2,제주특별자치도,서귀포시,126578,85099,53035,25861,4532,0,83428,1671,41479


In [207]:
g6_jeju_22.to_csv("temp2_2_governor_jeju_6.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [208]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp2_2_governor_ulsan_6.csv
Added: temp2_1_governor_gangwon_6.csv
Added: temp2_1_governor_daejeon_6.csv
Added: temp2_1_governor_daegu_6.csv
Added: temp2_2_governor_chungnam_6.csv
Added: temp2_2_governor_daegu_6.csv
Added: temp2_1_governor_chungnam_6.csv
Added: temp2_2_governor_daejeon_6.csv
Added: temp2_1_governor_busan_6.csv
Added: temp2_2_governor_jeju_6.csv
Added: temp2_1_governor_gyeongbuk_6.csv
Added: temp2_2_governor_gyeonggi_6.csv
Added: temp2_1_governor_jeonbuk_6.csv
Added: temp2_1_governor_jeonnam_6.csv
Added: temp2_1_governor_jeju_6.csv
Added: temp2_2_governor_gangwon_6.csv
Added: temp2_2_governor_gwangju_6.csv
Added: temp2_1_governor_chungbuk_6.csv
Added: temp2_1_governor_gwangju_6.csv
Added: temp2_1_governor_seoul_6.csv
Added: temp2_1_governor_gyeonggi_6.csv
Added: temp2_2_governor_jeonbuk_6.csv
Added: temp2_2_governor_gyeongnam_6.csv
Added: temp2_1_governor_incheon_6.csv
Added: temp2_1_governor_sejong_6.csv
Added: temp2_1_governor_gyeongnam_6.csv
Added: temp2_2_gov